**Import Statements**

In [700]:
import os
import sys
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.sparse as ssp
import seaborn as sns; sns.set() 
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import preprocessing
from copy import deepcopy as dcopy
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import explained_variance_score,r2_score,accuracy_score 
from sklearn.ensemble.forest import _generate_unsampled_indices,_generate_sample_indices
from sklearn.ensemble import RandomForestClassifier

**Decalaration of initial file path for the results to be saved**

In [701]:
iterResultPath = './7_Cost_complexity_results/'

**Plot Function**
- To plot the size and accuracy raatio of the cost complexity pruned random forest

In [702]:
def plot_the_scores(_models, plot_Values, tree_no, datasets_used, filesName ):
            
    size_matrix, acc_matrix, alpha_matrix=plot_Values
    
    plt.figure()
    plt.xticks(range(len(_models)), _models)
    figure, axs = plt.subplots(nrows=2, ncols=1, sharex=True)
    axs[0].boxplot(size_matrix.T, labels=_models, showmeans=True, meanline=True)
    axs[0].set_ylabel('Ratio of the size')
    axs[0].set_ylim([0, 1])
    axs[1].boxplot(acc_matrix.T, labels=_models, showmeans=True, meanline=True)
    axs[1].set_ylabel('Ratio of the accuracy')
    axs[1].set_ylim([0, 2])
    #plt.('Size and accuracy ratio of #decisiontrees = '+repr(trees_no) + ', Datasets : ' + datasets_used)
    plt.savefig(filesName+'.png')
    plt.show()
    plt.close('all')
    
#     plt.figure()
    
#     plt.plot(alpha_matrix[0], label=r'$\alpha^{\ast}$'+'-DecisionTree')
#     plt.plot(alpha_matrix[1], label=r'$\alpha^{\ast}$'+'-RandomForest')
   
    
#     plt.ylabel('Cost-Complexity parameters :'+  r'$\alpha^{\ast}$')
#     plt.title('Averaged Optimal Cost-Complexity parameters')
#     plt.xlabel('Decision Tree index')
#     plt.legend(loc='best')
#     plt.savefig(filesName+'_alphas_Value'+'.png')
#     plt.show()
    
    return

**Functions help to predict output of random forest for classification task**

In [703]:
def predict_randomForest(nodeScore_, testLeaves_Id):
    tree_no = len(nodeScore_)
    test_size = testLeaves_Id[0].shape[0]
    classes_no = nodeScore_[0].shape[1]
    probability_ValTest = np.zeros(shape=(test_size, classes_no))
    for k in range(tree_no):
        probability_ValTest += nodeScore_[k][testLeaves_Id[k], :]
    probability_ValTest = probability_ValTest/tree_no
    y_prediction_Forest = np.argmax(probability_ValTest, axis=1)
    return y_prediction_Forest

**Prunes the initail decsion tree**
- Helps to prune to optimal subtree using the optimum alpha obtained after cross validation using oob and training dataset


In [704]:
def get_optprunedTree(randomForest, testLeaves_id,opt_alpha_value,min_alpha_value_list):
    trees_no = randomForest.n_estimators
    opt_nl_ = [0]*trees_no
    testLeaves_id_ = {}
    
    for k, estimator_ in enumerate(randomForest):
        
        testLeaves_id_[k] = testLeaves_id[k][opt_alpha_value[k]]
        unprunedLeaves_id = testLeaves_id[k][min_alpha_value_list[k]]
        orig_nl_ = len(np.unique(unprunedLeaves_id))
        opt_nl_[k] = len(np.unique(testLeaves_id_[k]))/orig_nl_

    return testLeaves_id_, np.mean(opt_nl_)

**Out of Bag dataset prediction**
- This method claculates the prediction of of oob sample 
- Input -Leaves id for each tree
- Node score


In [705]:
def predict_forestOob(nodeScore_,outOfBag_indices, outOfBag_leaves_id,train_size):
    trees_no = len(nodeScore_)
    num_classes = nodeScore_[0].shape[1]
    probOob_ = np.zeros(shape=(train_size, num_classes))
    for t in range(trees_no):
        probOob_[outOfBag_indices[t], :] += nodeScore_[t][outOfBag_leaves_id[t],:]
    y_pred_outOfBa = np.argmax(probOob_, axis=1)
    print(y_pred_outOfBa,"---y_pred_outOfBa")
    return y_pred_outOfBa

**Global optimal cost complexity parameter method**
- Function returns optimal gloabl cost complexity parameter alpha for all the decsion trees .
- Possible  while globally pruning the random forest

In [706]:
def get_globOptAlpha(randomForest, y_train, outOfBag_indices, alphaOutOfBag_leaves_id,alpha_value_list, nodeScore_):
    uniqueAplha = []
    print('')
    print(alpha_value_list,"---alpha_value_list")
    
    #To convert into matrix format with n rows and columns
    for k in alpha_value_list:
        
        uniqueAplha = uniqueAplha + alpha_value_list[k]
        alpha_value_list[k] = np.array(alpha_value_list[k])
        
    uniqueAplha = np.array(uniqueAplha)
    print('')
    print(uniqueAplha,"---uniqueAplha")
    
    #Internal parameter so as to minimize alpha disscretization
    numDecimals = 10
    uniqueAplha = np.round(uniqueAplha, numDecimals)
    uniqueAplha = np.unique(uniqueAplha)
    trainSize = len(y_train)
    
    acc_ValueList_test = []
    levelIndex = {}
    
    for k, estimator in enumerate(randomForest):
        levelIndex[(0,k)] = 0
    print(levelIndex,"---levelIndex") 
    
    
    #To return leave id value of threshold
    def get_oobLeavesId_at_alpha(alp_thresh):       
            
        outOfBag_leaves_id = {}
        alphaKeys = []
        
        for t_, estimator in enumerate(randomForest):
            
            alphaVal = alpha_value_list[t_][alpha_value_list[t_] <= alp_thresh]
            
            #When alphaVal is empty value it returns the largest alpha 
            if(sum(alphaVal)): 
                alpha_t_ = max(alphaVal)
            
            #Otherwise it retruns full tree ie lowest alpha  val
            else:
                alpha_t_ = min(alpha_value_list[t_])
                
            outOfBag_leaves_id[t_] = alphaOutOfBag_leaves_id[t_][alpha_t_]
            alphaKeys.append(alpha_t_)
            
        return outOfBag_leaves_id, alphaKeys
    
    allAlphas = {}
    
    for k, alp in enumerate(uniqueAplha):
        
        outOfBag_leaves_id, allAlphas[k] = get_oobLeavesId_at_alpha(alp)
        y_pred_outOfBag =predict_forestOob(nodeScore_,outOfBag_indices, outOfBag_leaves_id,trainSize)
        acc_ = get_randomForest_acc(y_train, y_pred_outOfBag)
        acc_ValueList_test.append(acc_)

        
    opt_alphaOob = allAlphas[np.argmax(acc_ValueList_test)]
        
    return opt_alphaOob

**Predict Method**
- Used the tree for classification task

In [707]:
def predictTree(nodeScore_, leavesId):
    p_val = nodeScore_[leavesId, :]
    y_predicted_value = np.argmax(p_val, axis=1)
    return y_predicted_value

**Optimum alpha  in cost complexity pruning**
- This method  calculatesthe  optimum alpha  value for list of  trees given dataset for crossvalidation 

In [708]:

def get_optAlpha_(randomForest, outOfBag_leaves_id, y_train, alpha_value_list, outOfBag_indices, 
                              nodeScore_):
    optimum_alpha = []
    for k, estimator in enumerate(randomForest):
        
        testAcc_list = []
        
        for alp in alpha_value_list[k]:
            
            leavesId = outOfBag_leaves_id[k][alp]
            y_predictValue = predictTree(nodeScore_[k], leavesId)
            
            print(k,estimator,"---k,estimator")
            print(leavesId,"---leavesId")
            print(y_predictValue,"---y_predictValue")
            
            #Get accuracy
            accuracy_ = get_randomForest_acc(y_train[outOfBag_indices[k]], y_predictValue)
            testAcc_list.append(accuracy_)
       
        print(testAcc_list,"---testAcc_list")
        optimum_index = np.argmax(testAcc_list)
        print(testAcc_list,"---testAcc_list")
        optimum_alpha.append(alpha_value_list[k][optimum_index])
        print(optimum_alpha,"----optimum_alpha")
    
    print('')    
    print(optimum_alpha,"----optimum_alpha") 
    return optimum_alpha

**Prune Internal Node**    
- This method help to prune the  internal  node  coressponding to specified  node id both from left and right children list.  

In [709]:
def prune_value_list(k, c_left, c_right):
    cc_left = list(c_left)
    cc_right = list(c_right)
    stack_source = [k]
    while len(stack_source)>0:
        k = stack_source.pop()
        if(cc_left[k]!=cc_right[k]):
            stack_source.append(cc_left[k])
            stack_source.append(cc_right[k])
            cc_left[k] = -1
            cc_right[k] = -1
    return cc_left, cc_right

**Cost Complexity parameter**
- This method returns cost complexity parameter (alpha) corresponding to the nodes in the tree

In [710]:
def calculate_alphaValue(impurity_rate, n_node_sample_values, internal_nodes_, c_left, c_right):
    
    alpha_value = {}
    #print(internal_nodes_,"---internal_nodes_")
    
    #Iterate for each internal node in the tree
    for node_ in internal_nodes_:
        
        leaf_node, __ = get_Leaves_below_node(node_, c_left, c_right)
        #print(leaf_node,"---leaf_node")
        RT = impurity_rate[node_] * n_node_sample_values[node_]
        #print(RT,"---RT")
        RTt = np.dot(impurity_rate[leaf_node], n_node_sample_values[leaf_node])
        #print(RTt,"---RTt")
        alpha_value[node_] = (RT - RTt)/ (len(leaf_node)-1)
        
    #print(alpha_value,"---alpha_value")  
    return alpha_value

**Leaves at node n**
- This method return the respected nodes and leaves below the specified node k 

In [711]:
def get_Leaves_below_node(k, c_left, c_right):
    
    #print(c_left,"---c_left")
    #print(c_right,"---c_right")
    stackspace = [k]
    n_leaf = []
    i_node = []
    
    while len(stackspace) > 0:
        k = stackspace.pop()
        #print(k,"---k")

        if (c_left[k] != c_right[k]):
            #print(c_left[k],"---c_left[k]")
            #print(c_right[k],"---c_right[k]")
            stackspace.append(c_left[k])
            stackspace.append(c_right[k])
            #print(stackspace,"---stackspace")
            i_node.append(c_left[k])
            i_node.append(c_right[k])
            #print(i_node,"---i_node")
            
        else:
            
            n_leaf.append(k) 
            
        i_node = list(set(i_node)-set(n_leaf))
    #print(i_node,"---Final i_node")
    #print(n_leaf,"---Final n_leaf")
    #Returns  all the leaf nodes and other nodes seperately
    return n_leaf, i_node

**Prune the tree**
- Prunes current tree with the help of input score and id corresponding to each leaf
        - Tree structure and tree leaves id are given as input
        - Returns pruned tree leaves id

In [712]:
def pruneTree_(treeParameters_, outOfBag_id, testId_):
    
    c_left, c_right, impurity_rate, n_node_sample_values=treeParameters_
    
    #To get all the nodes below 0th node
    leaves_nodes, i_nodes_ = get_Leaves_below_node(0, c_left, c_right)
    
    #Initialization
    alpha_value_list, node_value_list = [], []
    all_outOfBag_id, all_testId_ = {}, {}
    temp_root_outOfBag = dcopy(outOfBag_id)
    temp_rootTest_ = dcopy(testId_)
    
    
    while(len(i_nodes_)):
            
            #Calculates alpha value corresponding to each node
            alpha_value = calculate_alphaValue(impurity_rate, n_node_sample_values, i_nodes_, c_left, c_right)
            print(alpha_value,"--- Returned alpha_value")
            
            if(alpha_value):
                
                min_alphaNode = min(alpha_value, key=alpha_value.get)
                print(min_alphaNode,"---min_alphaNode")
                min_alpha_value = alpha_value[min_alphaNode]
                print(min_alpha_value,"---min_alpha_value")
                
                alpha_value_list.append(min_alpha_value)
                node_value_list.append(min_alphaNode)
                
                c_left_, c_right_ = prune_value_list(min_alphaNode, c_left, c_right)
                leaves_nodes_, i_nodes_ = get_Leaves_below_node(0,c_left_, c_right_)
                
                #This section can be utilized to label the test samples
                for lll in leaves_nodes_:
                    old_leaves, _ = get_Leaves_below_node(lll,c_left, c_right)
                    
                    for leaf_ in old_leaves:
                        outOfBag_id[outOfBag_id==leaf_] = lll
                        testId_[testId_==leaf_] = lll
                        
                all_outOfBag_id[min_alpha_value] = dcopy(outOfBag_id)
                all_testId_[min_alpha_value] = dcopy(testId_)
                
                c_left, c_right = c_left_, c_right_
            else:
                print('Done with pruning')
                break
                
    #print(alpha_value,"---alpha_value outside the loop")
    eps = 1e-10 #Internal value of the parameter
    least_alpha_value = min(alpha_value_list) - eps
    all_outOfBag_id[least_alpha_value] = temp_root_outOfBag
    all_testId_[least_alpha_value] = temp_rootTest_
    alpha_value_list = [least_alpha_value] + alpha_value_list
    #print(alpha_value_list,"---alpha_value_list")
    #print(all_outOfBag_id,"---all_outOfBag_id")
    #print(all_testId_,"---all_testId_")
    return alpha_value_list, all_outOfBag_id, all_testId_


**Method to get class probabilities**
- This method return class probabilities of all nodes at once   

In [713]:
def get_class_pb(estimator_tree):
    #print(estimator_tree.tree_.value,"---estimator_tree.tree_.value")
    #print(estimator_tree.tree_.value[:,0,:],"---estimator_tree.tree_.value[:,0,:]")
    bin_cnt = estimator_tree.tree_.value[:,0,:]
    #print(bin_cnt,"---bin_cnt")
    bin_cnt /= bin_cnt.sum(axis=1)[:,np.newaxis]
    return bin_cnt

**Error Function**
- To get the impurity of each node in estimator tree

In [714]:
def get_errorFun_(estimator_tree):   
    return estimator_tree.tree_.impurity

**Aplha Sequence Calculator**
- Calculates the aplha value sequence with respect to the nested trees in each random forest.
- Return value-
        -Aplha_value_list: A list of alpha values  corresponding to each estimator
        -Training accuracy after each set of alpha value
        -List of nodes to be pruned according to alpha value

In [715]:
def get_alpha_value(randomForest, X_train, X_test, outOfBag_indices):
    
    nodeScore_, outOfBag_leaves_id, alpha_value_list = {}, {}, {}
    tree_no = randomForest.n_estimators
    min_alpha_value_list = {}
    testLeaves_id = {}
    
    for treeno in range(tree_no):
        estimator_tree = randomForest.estimators_[treeno]
        
        #To get the child nodes  of tthe decsion tree
        c_left = np.array(estimator_tree.tree_.children_left, dtype=int)
        c_right = np.array(estimator_tree.tree_.children_right, dtype=int)
        
        X_outofBag_tree = X_train[outOfBag_indices[treeno], :].view()
        #print(X_outofBag_tree,"---X_outofBag_tree")

        #To get the error function of the estimator to get the impurity    
        impurity_rate = get_errorFun_(estimator_tree)
        print(impurity_rate,"---impurity_rate")
        
        n_node_sample_values = estimator_tree.tree_.n_node_samples/estimator_tree.tree_.n_node_samples[0]
        #print(estimator_tree.tree_.n_node_samples,"---estimator_tree.tree_.n_node_samples")
        #print(estimator_tree.tree_.n_node_samples[0],"---estimator_tree.tree_.n_node_samples[0]")
        #print(n_node_sample_values,"---n_node_sample_values")
        
        #To get the probabilities of all node at once
        nodeScore_[treeno] = get_class_pb(estimator_tree)
        #print(nodeScore_[treeno],"---nodeScore_[treeno]")
       
        #Return the index of the leaf  for Out of bag sample set ends up in 
        outOfBag_id = estimator_tree.apply(X_outofBag_tree)
        print(outOfBag_id,"---outOfBag_id")
         
        #Return the index of the leaf  for test sample set ends up in 
        testId_ = estimator_tree.apply(X_test)
        print(testId_,"---testId_")
        
        treeParameters_ = c_left, c_right, impurity_rate, n_node_sample_values
        
        #To get the minimum value for alpha 
        alpha_value_list[treeno], outOfBag_leaves_id[treeno], testLeaves_id[treeno] = pruneTree_(treeParameters_, outOfBag_id, testId_)
        #print(alpha_value_list[treeno],"---alpha_value_list[treeno]")
        #print(outOfBag_leaves_id[treeno],"---outOfBag_leaves_id[treeno]")
        #print(testLeaves_id[treeno],"---testLeaves_id[treeno]")
        min_alpha_value_list[treeno] = min(alpha_value_list[treeno])
        #print(min_alpha_value_list[treeno],"---min_alpha_value_list[treeno]")
        
    return (alpha_value_list, outOfBag_leaves_id, testLeaves_id, nodeScore_, min_alpha_value_list)

**Method to get Out Of Bag indices for each estimator in random forest**
- This method returns a dictionary where key indicates estimator and value indicates the out of bag indices

In [716]:
def get_outOfBag_indices(randomForest, X_train):
    outOfBag_indices = {}
    for k, _estimator in enumerate(randomForest):
        #print(k,"k")
        outOfBag_indices[k] = _generate_unsampled_indices(_estimator.random_state,X_train.shape[0])
        #print(X_train.shape[0],"X_train.shape[0]")
        #print(outOfBag_indices[k],"outOfBag_indices[k]")
    #print(outOfBag_indices)
    return outOfBag_indices

**Method to claculates the accuracy**
- Return accuracy score of random forest 

In [717]:
def get_randomForest_acc(y_test, y_predict):
    rf_acc = accuracy_score(y_test, y_predict)
    return rf_acc

**Helper function**
- Helper Method is helps to run all the iterations in parralel loop

In [718]:
def helper_parallel_fuction(arguments):
    evaluating_func = arguments[0]
    evaluating_func_args = arguments[1:]
    return evaluating_func(*evaluating_func_args)

**Cost Complexity Pruning**
- Prunes the random forest using out of bag indices with cross validation

In [731]:
def costComplexityPruning_withCv(randomForest, data_for_processing, CV_set='OOB-set'):    
    
    #Data set
    X_train, X_test, y_train, y_test = data_for_processing
    #print("X_train",X_train,"X_test", X_test,"y_train", y_train,"y_test", y_test)
    
    #To get the accuracy scor of the original random forest
    randomForest_acc = get_randomForest_acc(y_test, randomForest.predict(X_test))
    print(randomForest_acc,"---randomForest_acc")
    
    #To get the out of the bag indices
    outOfBag_indices = get_outOfBag_indices(randomForest, X_train)
    print(outOfBag_indices,"---outOfBag_indices")
    
    #To get potential alpha value 
    alpha_param = get_alpha_value(randomForest, X_train, X_test, outOfBag_indices)
    print(alpha_param,"---alpha_param")
    alpha_value_list, outOfBag_leaves_id, testLeaves_id, nodeScore_, min_alpha_value_list = alpha_param 
    
    #To get Cross validation error for estimator by setting the optimum alpha[t] value
    opt_alpha_value = get_optAlpha_(randomForest, outOfBag_leaves_id, y_train, alpha_value_list, outOfBag_indices, 
                              nodeScore_)   
        
    #To get Cross validation error for forest by setting the optimum alpha value gloablly
    opt_alphaOob = get_globOptAlpha(randomForest, y_train, outOfBag_indices, outOfBag_leaves_id, 
                                       alpha_value_list, nodeScore_)
    print(opt_alphaOob,"---opt_alphaOob")
      
    #Prune the tree upto reaching optimal aplha value
    optiTestId, pruneRatio = get_optprunedTree(randomForest, testLeaves_id,opt_alpha_value,min_alpha_value_list)
    yPrune = predict_randomForest(nodeScore_, optiTestId)
    accuracy_test_pruned = get_randomForest_acc(y_test, yPrune)
    print(accuracy_test_pruned,"---accuracy_test_pruned")
    
    #Prune the Forest upto reaching optimal aplha value
    optiTestIdOob, pruneRatioOob = get_optprunedTree(randomForest, testLeaves_id,opt_alphaOob,min_alpha_value_list)
    y_pruneOob = predict_randomForest(nodeScore_, optiTestIdOob)
    accuracy_test_pruned_oob = get_randomForest_acc(y_test, y_pruneOob)
    print(accuracy_test_pruned_oob,"---accuracy_test_pruned_oob") 
    
    
    #To plot the alpha value
    def plot_alpha_value_lists():
        plt.figure()
        for k in alpha_value_list:
            plt.plot(alpha_value_list[k])
        plt.xlabel('No: of unique' +r'$\alpha value \in \mathcal{A}_j$')
        plt.ylabel('Cost-complexity param' + r'$\mathcal{A}_j$')
        plt.show()
        
    #plot_alpha_value_lists()
    
    
    #Ratio between pruned aand unpruned decsion tree
    accuracy_testRatio = accuracy_test_pruned/randomForest_acc
    accuracy_testRatio_oob = accuracy_test_pruned_oob/randomForest_acc
    print(accuracy_testRatio ,"---accuracy_testRatio  using test sample")
    print(accuracy_testRatio_oob,"---accuracy_testRatio_oob using oob sample")
    
    
    Finaloutput_array = np.array([pruneRatio, accuracy_testRatio, pruneRatioOob,
                             accuracy_test_pruned_oob,randomForest_acc])
    alpha_values = np.concatenate((opt_alpha_value, opt_alphaOob))
    
    return np.concatenate((Finaloutput_array, alpha_values))


**Initialize and fit random forest classifier with the training dataset**
- Bootstrap  is initialized as true and this method requires model name and dataset as input parameters
- Random forest classifier with n number of estimators is the expected output

In [720]:
def get_randomforest(model_name, tree_no, data_set):
    X_train, X_test, y_train, y_test = data_set
    randomForest = RandomForestClassifier(bootstrap=True,n_estimators = tree_no)
    randomForest.set_params()
    randomForest.fit(X_train, y_train)
#     print(randomForest)
    return randomForest

**To read the dataset from the datasource**
- No input parameters required
- Outputs data in the form of attributtes and class form

In [721]:
def get_irisdata():
    irisdata = datasets.load_iris()
    X = irisdata.data  
    y = irisdata.target
#     print(X,y,"---X,y")
    return X, y

**This method calls the method to get the dataaset**
- Multiple dataset are retrieved from this method. Required input is the name of indented dataset
- Outputs dataset that are ready for test train split

In [722]:
def get_dataset(name):
#     print(name,"---name")
    dataset_method = {"iris_dataset" : get_irisdata()}  
#     print(dataset_method[name],"---dataset_method[name]")
    return dataset_method[name]

**Splits the dataset into test and train data**
- Input required- The split size of the test data and the dataset to be split
- Output- Provides 2 pair of dataset - Train and test data

In [723]:
def get_data(data_set, split_size):
    X, y = get_dataset(data_set)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=split_size)
    data_for_processing = X_train, X_test, y_train, y_test
#     print(data_for_processing,"---data_for_processing")
    return data_for_processing

**Methof to specify Split Size**
- This method specifies the splt size of each dataset
- Outputs split size along with the dataset name

In [724]:
def get_split_size(dataset_name):
    test_size={'iris_dataset': 0.25}
#     print(test_size,"---test_size")
    return test_size[dataset_name]

**To test the random forest classifier on out of bag set and train set**
- Both these oob and train dataset are considered as cross validation set and compared
- Required Inputs- Cross validation set i.e out of bag dataset and train dataset

In [744]:
def test_cc_prune(iter_no,test_models,datasets_used,tree_no,CV_set='OOB-set'):
    
    #To create the Folders if it doesnt exist
    if not os.path.exists(iterResultPath):
        os.makedirs(iterResultPath)
        
    #To open the text file for writing the results obtained
    stdout_ = sys.stdout
    file = open('./7_Cost_complexity_results/Results__.txt', 'w')
    sys.stdout = file    
        
    #Iterate through each dataset
    for data_set in datasets_used:        
        
        #To get the test train split ratio of each dataset
        print(data_set,"---data_set")
        split_size = get_split_size(data_set)        
        print(split_size,"---split_size")
        
        #Initializing matrix
        size_matrix = np.zeros(shape=(len(test_models)*2,iter_no))
        acc_matrix = np.zeros(shape=(len(test_models)*2,iter_no))
        rf_matrix = np.zeros(shape=(len(test_models)*2,iter_no))
        alpha_matrix = np.zeros(shape=(len(test_models)*2,iter_no, tree_no))
        print(size_matrix,"---size_matrix")
        print(acc_matrix,"---acc_matrix")
        print(alpha_matrix,"---alpha_matrix")
        
        i = 0
        _models = []
        output_dict = {}
        for m, model in enumerate(test_models):
            print(test_models,"---test_models")
            print(m,"---m")
            print(model,"----model")
            argument_list = []
            for k in range(iter_no):
                #Get the data as train and test dataset for further processing
                data_for_processing = get_data(data_set, split_size)
                #Create Random forest classifier object using the dataset and given parameters
                randomForest = get_randomforest(model, tree_no, data_for_processing)                          
                #To create Tuple to run parallel
                argumentTuple = (costComplexityPruning_withCv,randomForest, data_for_processing, CV_set)
                argument_list.append(argumentTuple)         
                
#                 for arguments in argument_list:            
#                     evaluating_func = arguments[0]                    
#                     evaluating_func_args = arguments[1:]
#                     results_of_dataset=evaluating_func(*evaluating_func_args)
                
            results_of_dataset = Parallel(n_jobs=4)(delayed(helper_parallel_fuction)(arguments) for arguments in argument_list)
            print(results_of_dataset,"---results_of_dataset")
            results_matrix = np.array(results_of_dataset) 
            print(results_matrix,"---results_matrix")

            size_matrix[i,:] = results_matrix[:,0]
            acc_matrix[i,:] = results_matrix[:,1]
            
            
            value1_mean = "{0:.3f}".format(results_matrix[:,0].mean())
            value1_std =  "{0:.3f}".format(results_matrix[:,0].std())
            value1 =  value1_mean + '+/-' + value1_std
            print(value1,"--Size -value1")
            
            value2_mean = "{0:.3f}".format(results_matrix[:,1].mean()) 
            value2_std = "{0:.3f}".format(results_matrix[:,1].std()) 
            value2 = value2_mean + '+/-' + value2_std
            print(value2,"---Accuracy value2")
            
            
            output_dict[model+'-DecsionTree'] = value1 +',   '+ value2
            print(output_dict,"---output_dict")
            
            
            alpha_matrix[i,:,:] = results_matrix[:,5:5+tree_no]
            print(alpha_matrix,"---alpha_matrix")
            
            i+=1
            
            _models.append(model+'\n'+'DecsionTree')
            print(_models,"---_models")
            
            size_matrix[i,:] = results_matrix[:,2]
            acc_matrix[i,:] = results_matrix[:,3]
            
            rf_matrix[i,:]= results_matrix[:,4]
            print(rf_matrix,"---rf_matrix")
            
            
            
            value1_mean = "{0:.3f}".format(results_matrix[:,2].mean()) 
            value1_std = "{0:.3f}".format(results_matrix[:,2].std())
            value1 = value1_mean + '+/-' + value1_std
            print(value1,"---Size value1")
            
            value2_mean = "{0:.3f}".format(results_matrix[:,3].mean()) 
            value2_std = "{0:.3f}".format(results_matrix[:,3].std()) 
            value2 = value2_mean + '+/-' + value2_std
            print(value2,"---Accuracy value2")
            
            output_dict[model+'-RandomForest'] = value1 +',   '+ value2
            print(output_dict,"---Second output_dict")
            
            alpha_matrix[i,:,:] = results_matrix[:,5+tree_no::]
            print(alpha_matrix,"---alpha_matrix")
            
            i+=1
            
            _models.append(model+'\n'+'OOB')
            print(_models,"---Second _models")

        plot_Values = size_matrix, acc_matrix, alpha_matrix
        print(size_matrix,"---size_matrix")
        print(acc_matrix,"---acc_matrix")
        print(alpha_matrix,"---alpha_matrix")
        
        print('')
        print(datasets_used)
        print('-----------------------------------------------')
        for k in output_dict:
            print(k + '==> Size, Accuracy ratio  : ' + output_dict[k])             

    sys.stdout = stdout_
    file.close()
    return

In [749]:
test_cc_prune_param={'test_models' : ['RandomForestClassifier'],
                                 'datasets_used' : ['iris_dataset'],
                                 'iter_no' : 1,
                                 'tree_no' : 500}

In [750]:
print(test_cc_prune_param)
test_cc_prune(**test_cc_prune_param)